In [1]:
import pandas as pd
from binance.client import Client

In [145]:
def get_depth(symbol):
    client = Client()
    depth = client.get_order_book(symbol=symbol,limit=5000)
    bid = pd.DataFrame(depth['bids'], columns=['price','volume']).astype(float)
    bid['volume'] = bid['volume']*bid['price']
    ask = pd.DataFrame(depth['asks'], columns=['price','volume']).astype(float)
    ask['volume'] = ask['volume']*ask['price']

    #Precio promedio
    avg_price = (bid.iloc[0]['price'] + ask.iloc[0]['price'] ) / 2
    
    #Filtra los precios hasta un +/-15% en referencia al avg_price
    bid = bid[bid['price'] > avg_price*0.85] 
    ask = ask[ask['price'] < avg_price*1.15] 

    #agrega un indice del tipo fecha para hacer el resample
    bid.index = pd.date_range(start='2023-01-01', periods=len(bid), freq='T')
    ask.index = pd.date_range(start='2023-01-01', periods=len(ask), freq='T')

    #Calculando la cantidad de periodos a resamplear
    T = ask[ask['price']<(avg_price*1.005)]['price'].count()
    T = 1000 if T > 1000 else T
    period = f'{T}T'

    #Resample, asignacion de indice por Price y agregar columna Perc (% en relacion al precio promedio)
    grouped_bid = bid.resample(period).agg({'price': 'mean', 'volume': 'sum'})
    grouped_bid['perc'] = round(((grouped_bid['price']/avg_price)-1)*100,1)
    grouped_bid.set_index('price', inplace=True)
    grouped_ask = ask.resample(period).agg({'price': 'mean', 'volume': 'sum'})
    grouped_ask['perc'] = round(((grouped_ask['price']/avg_price)-1)*100,1)
    grouped_ask.set_index('price', inplace=True)

    #Unificando datoso de Bid y ask en Depth
    depth = pd.DataFrame(columns=['volume', 'perc'])
    depth = pd.concat([depth, grouped_bid])
    depth = pd.concat([depth, grouped_ask])
    
    avg_volume = depth['volume'].sum()
    depth['volume_perc'] = round((depth['volume']/avg_volume)*100,1)

    depth = depth.sort_values(by='volume', ascending=False)
    return depth


In [146]:
depth = get_depth('LUNCUSDT')
print(depth[depth['perc']>0].head(3))
print(depth[depth['perc']<0].head(3))


                 volume  perc  volume_perc
0.000180  472752.582177  14.3          9.6
0.000158  471764.011015   0.2          9.5
0.000175  455986.548581  11.1          9.2
                 volume  perc  volume_perc
0.000157  571270.558584  -0.3         11.5
0.000140  376439.142782 -11.3          7.6
0.000150  210927.502881  -5.0          4.3
